In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp plugin.authenticators.oauth

In [ ]:
# export 
# hide
import abc
from time import sleep
from pymemri.plugin.states import RUN_USER_ACTION_NEEDED, RUN_USER_ACTION_COMPLETED

## OAuth Authenticator

OAuth Authenticator provides an easy interface to setup standard Open Authorization (OAuth) with 3rd party services.

Simply calling `authenticate()` should load the related account with required credentials.

Inheriting class should implement:
- get_oauth_url() to setup OAuth endpoint
- get_tokens_from_code() to retrieve tokens from the service with returned OAuth code
- refresh_tokens() to get new token pairs


In [ ]:
# export
# hide

class OAuthAuthenticator(metaclass=abc.ABCMeta):

    SLEEP_INTERVAL = 1.0

    def __init__(self, client, pluginRun):
        self.client = client
        self.pluginRun = pluginRun
        self.isTest = False

    def set_test(self, isTest):
        self.isTest = isTest

    def authenticate(self, test=False):

        tokens = None
        if self.pluginRun.account[0].refreshToken:
            try:
                tokens = self.refresh_tokens(self.pluginRun.account[0].refreshToken)
            except:
                pass

        if not tokens:
            url = self.get_oauth_url()
            code = self.present_url_to_user(url)
            tokens = self.get_tokens_from_code(code)

        self.store_tokens(tokens)

    def present_url_to_user(self, url, test=False):
        if self.isTest:
            return 'dummy_code'
        # request user to visit url
        self.pluginRun.oAuthUrl = url
        self.pluginRun.state = RUN_USER_ACTION_NEEDED
        self.pluginRun.update(self.client)
       # WAIT HERE = BLOCK
        while True:
            sleep(self.SLEEP_INTERVAL)
            self.pluginRun.reload(self.client)
            if self.pluginRun.state == RUN_USER_ACTION_COMPLETED:
                return self.pluginRun.account[0].code

    def store_tokens(self, tokens):
        self.pluginRun.account[0].secret = tokens['access_token']
        self.pluginRun.account[0].refreshToken = tokens['refresh_token']
        self.pluginRun.account[0].update(self.client)

    @abc.abstractmethod
    def get_oauth_url(self):
        raise NotImplemented()

    @abc.abstractmethod
    def get_tokens_from_code(self, code):
        """ Gets access and refresh tokens from 3rd party service
            and returns them in form:
                {
                    'access_token': '...',
                    'refresh_token': '...'
                }
        """
        raise NotImplemented()

    @abc.abstractmethod
    def refresh_tokens(self):
        """ Gets new tokens by using an existing refresh token
            and returns them in form:
                {
                    'access_token': '...',
                    'refresh_token': '...'
                }

        """
        # use self.pluginRun.account[0].refreshToken
        raise NotImplemented()


In [ ]:
# export
# hide

class ExampleOAuthAuthenticator(OAuthAuthenticator):

    def get_oauth_url(self):
        return "https://example.com/oauth"

    def get_tokens_from_code(self, code):
        return {
            'access_token': 'dummy_access_token',
            'refresh_token': 'dummy_refresh_token'
        }

    def refresh_tokens(self, refreshToken):
        return {
            'access_token': 'refreshed_dummy_access_token',
            'refresh_token': 'refreshed_dummy_refresh_token'
        }
